Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).


In [1]:
import pandas as pd
import altair as alt

In [2]:
df = pd.read_excel('2014-2020.xlsx')
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [3]:
df = df.drop(columns=['Unnamed: 12'])

In [4]:
df[['Hour','Date']] = df['Час/Дата'].str.split('-',expand=True)

In [5]:
df[['Day','Month','Year']] = df['Date'].str.split('.',expand=True)

In [6]:
df['Hour'] = pd.to_numeric(df['Hour'])
df['Day'] = pd.to_numeric(df['Day'])
df['Month'] = pd.to_numeric(df['Month'])
df['Year'] = pd.to_numeric(df['Year'])
df['time'] = pd.to_timedelta(df['Hour'], unit='h')
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df.drop(['Час/Дата', 'GAES_PUMP','UK_BLR_RUS', 'UK_EURO', 'UK_MLD'],  axis = 1, inplace = True)

In [8]:
df['Date_Time'] = df['Date'] +  df['time']

In [9]:
df.head()

,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,Hour,Date,Day,Month,Year,time,Date_Time
0,9235,2039,621,4942,385.0,0,16693,24,2020-12-31,31,12,2020,1 days 00:00:00,2021-01-01 00:00:00
1,9221,2159,707,5549,470.0,0,17805,23,2020-12-31,31,12,2020,0 days 23:00:00,2020-12-31 23:00:00
2,9249,2377,709,5906,1000.0,0,18870,22,2020-12-31,31,12,2020,0 days 22:00:00,2020-12-31 22:00:00
3,9256,2499,702,6329,909.0,322,19887,21,2020-12-31,31,12,2020,0 days 21:00:00,2020-12-31 21:00:00
4,9213,2521,702,6640,823.0,602,20387,20,2020-12-31,31,12,2020,0 days 20:00:00,2020-12-31 20:00:00


In [10]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [11]:
df1 = df.iloc[:, : 6]
df1['Year'] = df['Year']

In [12]:
df1 = df1.groupby('Year').sum()
df1 = df1.reset_index()
df1.head()

,Year,AES,TEC,VDE,TES,GES,GAES_GEN
0,2014,88204418,14684731,1606386,68605877,7991074.0,839842
1,2015,87413763,12041074,1234936,50259819,4964478.0,1554857
2,2016,80762094,12841033,1093190,49879543,7113989.0,1621377
3,2017,85314258,12208062,1177032,44945303,8531952.0,1575099
4,2018,84351328,12261018,1863726,47747092,10072767.0,1564867


### 1. Як змінювалась структура генерації електроенергії за роками?

Я вирішила обрати візуалізацію за допомогою площин, щоб відповісти на це запитання.
За допомогою такого типу графіку можна простежити сумарну генерацію електроенергії за різні роки, а також помітити окремий внесок кожного джерела до загальної генерації. Так ми бачимо і зміну в часі і частку від цілого.

Недоліком цього варіанту є те, що може бути важко простежити саме в якій кількості змінювалась генерація кожного із джерел.

Також тут зручно було б використовувати Bar Chart.

In [14]:
c = alt.Chart(df1).transform_fold(
    ['AES', 'TEC', 'VDE', 'TES', 'GES','GAES_GEN'], as_ = ['Source', 'Amount']).mark_area().encode(
    alt.X('Year:N', scale = alt.Scale(0)),
    alt.Y('Amount:Q', title = 'Amount of Power Generation'),
    alt.Color('Source:N'),
    tooltip = alt.Tooltip('Amount:Q')
).properties(title='Зміна структури генерації електроенергії за роками', width = 950, height = 500, background = '#F9F9F9', padding = 25)

In [15]:
c.show()

Displaying chart at http://localhost:18632/


In [16]:
df2_1 = df[['Date_Time', 'CONSUMPTION']]

In [17]:
df2_1

,Date_Time,CONSUMPTION
0,2021-01-01 00:00:00,16693
1,2020-12-31 23:00:00,17805
2,2020-12-31 22:00:00,18870
3,2020-12-31 21:00:00,19887
4,2020-12-31 20:00:00,20387
...,...,...
61363,2014-01-01 05:00:00,18072
61364,2014-01-01 04:00:00,18453
61365,2014-01-01 03:00:00,19107
61366,2014-01-01 02:00:00,19665


In [18]:
df2 = df[['Hour', 'CONSUMPTION']]
df2['day_of_year'] = df['Date_Time'].dt.dayofyear

/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
df_2 = df2.groupby(['Hour', 'day_of_year']).mean()['CONSUMPTION'].reset_index() 

In [20]:
df_2

,Hour,day_of_year,CONSUMPTION
0,1,1,18202.428571
1,1,2,17965.714286
2,1,3,16941.714286
3,1,4,15692.428571
4,1,5,13415.285714
...,...,...,...
8779,24,362,17921.000000
8780,24,363,17909.000000
8781,24,364,17889.000000
8782,24,365,18243.428571


### 2. Як залежить споживання електроенергії від дня року та години доби?

Оскільки в цьому завданні потрібно було відображати таку велику кількість даних як споживання електроенергії впродовж дня року та години доби, то найкращий варіант для мене була візуалізація Heat Map. 
За допомогою кольору легко відслідкувати значну зміну в споживанні.
Мінусом такої візуалізації є те, що неможливо побачити точну кількість спожитої електроенергії за конкретний день.

In [21]:
c2 = alt.Chart(df_2).mark_rect().encode(
    alt.X('day_of_year:O', title='Day of the year'),
    alt.Y('Hour:O', title='Hour of day'),
    alt.Color('CONSUMPTION:Q', title='Consumption', scale = alt.Scale(scheme = 'blueorange'))
).properties(width = 1250, height = 500, background = '#F9F9F9', padding = 25,
              title="Залежність споживання електроенергії від дня року та години доби")

In [22]:
c2.show()

Displaying chart at http://localhost:18632/


In [23]:
df3 = df.iloc[:, : 6]
df3['Hour'] = df['Hour']

df3 = df3.groupby('Hour').mean()
df3 = df3.reset_index()
df3.head()

,Hour,AES,TEC,VDE,TES,GES,GAES_GEN
0,1,9533.777865,1428.560813,164.178334,4883.335549,413.657020,0.26711
1,2,9530.698084,1424.868987,164.754009,4772.738365,325.804849,0.00000
2,3,9530.246774,1425.223309,164.704341,4722.442315,314.622214,0.00000
3,4,9531.818537,1423.511928,164.179898,4684.545561,289.973797,0.00000
4,5,9535.267110,1425.863512,165.059836,4697.965976,323.253813,0.00000


### 3. Зміна генерації електроенергії з різних джерел впродовж доби

В цьому випадку я вирішила обрати Bar Chart. За допомогою стовпчиків можна легко зобразити скільки загалом генерується електроенергії в різний час доби.
Зручно порівнювати кількість генерованої енергії з різних джерел впродовж дня. Знову ж таки, як і в першому завданні можна побачити зміну в часі та частики в цілому. Мінусом є те, що може бути важко порівнювати кількість генерації енергії певного джерела в різні години.

Як і в першому завданні можна також було використовувати Area plot.

In [24]:
c3 = alt.Chart(df3).transform_fold(
    ['AES', 'TEC', 'VDE', 'TES', 'GES','GAES_GEN'], as_ = ['Source', 'Amount']).mark_bar().encode(
    alt.X('Hour:N'),
    alt.Y('Amount:Q'),
    alt.Color('Source:N'),
    tooltip = alt.Tooltip('Amount:Q')
).properties(title="Зміна генерації електроенергії з різних джерел впродовж доби", width = 950, height = 500, background = '#F9F9F9', padding = 25)

In [25]:
c3.show()

Displaying chart at http://localhost:18632/


### 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

Щоб відповісти на дане питання, я зробила два графіки: Heat map впродовж доби у розрізі місяців року та впродовж доби у розрізі пір року.
Другий графік певним чином узагальнює перший. 
Також це запитання дуже схоже до другого завдання, саме тому для мене було зручно використовувати знову такий тип візуалізації.

In [26]:
df4 = df[['Date_Time','Hour','CONSUMPTION']]
df4['month_name'] = df['Date_Time'].dt.month_name()
df4

/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date_Time,Hour,CONSUMPTION,month_name
0,2021-01-01 00:00:00,24,16693,January
1,2020-12-31 23:00:00,23,17805,December
2,2020-12-31 22:00:00,22,18870,December
3,2020-12-31 21:00:00,21,19887,December
4,2020-12-31 20:00:00,20,20387,December
...,...,...,...,...
61363,2014-01-01 05:00:00,5,18072,January
61364,2014-01-01 04:00:00,4,18453,January
61365,2014-01-01 03:00:00,3,19107,January
61366,2014-01-01 02:00:00,2,19665,January


In [27]:
df4_1 = df4
df4_1['season'] = '-'

/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
for i in range(len(df4_1['month_name'])):
    if df4_1['month_name'][i] == 'January' or df4_1['month_name'][i] == 'February' or df4_1['month_name'][i] == 'December':
        df4_1['season'][i] = 'Winter'
    elif df4_1['month_name'][i] == 'March' or df4_1['month_name'][i] == 'April' or df4_1['month_name'][i] == 'May':
        df4_1['season'][i] = 'Spring'
    elif df4_1['month_name'][i] == 'June' or df4_1['month_name'][i] == 'July' or df4_1['month_name'][i] == 'August':
        df4_1['season'][i] = 'Summer'
    elif df4_1['month_name'][i] == 'September' or df4_1['month_name'][i] == 'October' or df4_1['month_name'][i] == 'November':
        df4_1['season'][i] = 'Autumn'

/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/alina/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [29]:
#df4_1 = df4.groupby(['Hour', 'season']).mean()['CONSUMPTION'].reset_index()
df4_1

,Date_Time,Hour,CONSUMPTION,month_name,season
0,2021-01-01 00:00:00,24,16693,January,Winter
1,2020-12-31 23:00:00,23,17805,December,Winter
2,2020-12-31 22:00:00,22,18870,December,Winter
3,2020-12-31 21:00:00,21,19887,December,Winter
4,2020-12-31 20:00:00,20,20387,December,Winter
...,...,...,...,...,...
61363,2014-01-01 05:00:00,5,18072,January,Winter
61364,2014-01-01 04:00:00,4,18453,January,Winter
61365,2014-01-01 03:00:00,3,19107,January,Winter
61366,2014-01-01 02:00:00,2,19665,January,Winter


In [30]:
c4 = alt.Chart(df4).mark_rect().encode(
    x = alt.X('month(Date_Time):O'),
    y = alt.Y('hours(Date_Time):O'),
    color = alt.Color('CONSUMPTION:Q', title='Consumption', scale = alt.Scale(domain =[17000,27000])),
    tooltip = alt.Tooltip('CONSUMPTION:Q')
).properties(title="Зміна споживання електроенергії впродовж доби у розрізі місяців року",
             width = 950, height = 500, background = '#F9F9F9', padding = 25)

In [31]:
c4.show()

Displaying chart at http://localhost:18632/


In [32]:
c4_1 = alt.Chart(df4_1).mark_rect().encode(
    x = alt.X('season:O'),
    y = alt.Y('hours(Date_Time):O'),
    color = alt.Color('CONSUMPTION:Q', title='Consumption', scale = alt.Scale(domain = [17000,27000])),
    tooltip = alt.Tooltip('CONSUMPTION:Q')
).properties(title="Зміна споживання електроенергії впродовж доби у розрізі пір року",
             width = 950, height = 500, background = '#F9F9F9', padding = 25)

In [33]:
c4_1.show()

Displaying chart at http://localhost:18632/


### 5. Як змінюється споживання електроенергії впродовж тижня?

Для цього завдання, я обрала візуалізацію Bar Chart.  
За допомогою стовпчиків дуже зручно порівнювати кількість споживаня енергії в різні дні протягом всього тижня.

Також можна було використовувати звичайний лінійний графік, проте за допомогою Bar Chart зручніше прослідкувати точну кількість спожитої електроенергії в конкретний день.


In [35]:
df5 = df[['CONSUMPTION']]
df5['weekday'] = df['Date_Time'].dt.day_name()
df5['weekday_num'] = df['Date_Time'].dt.weekday

/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/alina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
df5 = df5.groupby('weekday').mean()
df5 = df5.reset_index()
df5

,weekday,CONSUMPTION,weekday_num
0,Friday,17222.356923,4.0
1,Monday,17107.303425,0.0
2,Saturday,16823.421575,5.0
3,Sunday,16550.352169,6.0
4,Thursday,17293.883766,3.0
5,Tuesday,17230.552511,1.0
6,Wednesday,17274.907093,2.0


In [37]:
c5 = alt.Chart(df5).mark_bar().encode(
    alt.X('weekday:O', title='Day of week', sort = alt.Sort(field = 'weekday_num', order = 'ascending')),
    alt.Y('CONSUMPTION:Q', title='Consumption', 
          scale = alt.Scale(zero = False)),
     color = alt.Color('CONSUMPTION:Q', title='Consumption', scale = alt.Scale(scheme = 'greenblue')),
    tooltip = alt.Tooltip('CONSUMPTION:Q')
).properties(title="Зміна споживання електроенергії впродовж тижня", width = 950, height = 500, background = '#F9F9F9', padding = 25)

In [38]:
c5.show()

Displaying chart at http://localhost:18632/
